In [31]:
import pandas as pd

filename = '../big_a_data/all.csv'
df = pd.read_csv(filename, converters={'code': str})
df = df[df['code'].isin(['000063', '002179', '002223', '600660', '600887', '601318'])]
print('Done', len(df))
df

Done 6


,code,name,行业,上市时间,最新,涨幅,股息率_2023
43,000063,中兴通讯,通信设备,19971118,29.12,0.94,2.35
688,002179,中航光电,电子元件,20071101,36.52,0.77,1.64
731,002223,鱼跃医疗,医疗器械,20080418,36.52,0.25,2.19
3370,600660,福耀玻璃,玻璃玻纤,19930610,49.17,-0.12,2.64
3565,600887,伊利股份,食品饮料,19960312,25.7,2.27,4.67
3724,601318,中国平安,保险,20070301,41.94,0.19,5.79


In [32]:
DAYS  = 10
DAYS_MEAN = [5, 10, 20, 30]

def process_delta(file_hist, file_process):
    df_hist = pd.read_csv(file_hist, converters = {'股票代码':str})
    df_process = pd.read_csv(file_process, converters = {'股票代码':str})
    num_new = len(df_hist) - len(df_process)
    if num_new == 0:
        return
    #print(file_process)
    list_col_hist = list(df_hist.columns)
    list_col_process = list(df_process.columns)
    list_col_gap = list_col_process[len(list_col_hist):]
    for col in list_col_gap:
        df_hist[col] = 0.0
    
    df_hist = df_hist[-num_new : ]
    df_process = pd.concat([df_process, df_hist], ignore_index = False)    

    for index in range(len(df_process) - num_new, len(df_process)):
        '''
        index - 10: d10
        ...
        index - 2: d2
        index - 1: d1
        index: 
            d-1, [index - 1, '收盘'] / curr
            d-2, [index - 2, '收盘'] / curr
            ...
            d-10,[index - 10, '收盘'] / curr
        '''
        curr_day_close = df_process.loc[df_process.index[index], '收盘']
        curr_day_high = df_process.loc[df_process.index[index], '最高']
        curr_day_low = df_process.loc[df_process.index[index], '最低']
        for d in range(1, DAYS + 1):
            pre_day_close = df_process.loc[df_process.index[index - d], '收盘']
            pre_day_high = df_process.loc[df_process.index[index - d], '最高']
            df_process.loc[df_process.index[index], f'd-{d}'] = round((pre_day_close/curr_day_close - 1) * 100, 2)
            df_process.loc[df_process.index[index - d], f'd{d}'] = round((curr_day_high/pre_day_close - 1) * 100, 2)

        #日均
        for mean in DAYS_MEAN:
            mean_sum = df_process['收盘'][index - mean + 1 : index + 1].sum()          
            df_process.at[index, f'{mean}日均'] = round(mean_sum/mean, 2)

    df_process.to_csv(file_process, index = False)

In [33]:
import os

for code in df['code']:
    print(code, end='\r')
    filename_hist = f'../big_a_data/hist_daily/{code}.csv'
    filename_process = f'./process_daily/{code}.csv'
    if os.path.exists(filename_process):
        process_delta(filename_hist, filename_process)
        continue
    df_hist_daily = pd.read_csv(filename_hist, converters={'股票代码':str})
    #debug
    #df_hist_daily = df_hist_daily[ : 10]
    #df_hist_daily = df_hist_daily[['开盘', '收盘', '最高']]

    for d in range(DAYS, 0, -1):
        df_hist_daily[f'd-{d}'] = 0.0
    for mean in DAYS_MEAN:
        df_hist_daily[f'{mean}日均'] = 0.0
    for d in range(1, DAYS + 1):
        df_hist_daily[f'd{d}'] = 0.0

    sum_pre = {mean:0.0 for mean in DAYS_MEAN}
    for index, row in df_hist_daily.iterrows():
        curr_day_close = row['收盘']
        curr_day_low = row['最低']

        #前DAYS日
        for d in range(1, DAYS + 1):
            if index >= d:
                temp = df_hist_daily.iloc[index - d]['收盘']
                df_hist_daily.at[index, f'd-{d}'] = round((temp/curr_day_close - 1) * 100, 2)
            else:
                break    
                
        #后DAYS日
        buy = curr_day_close
        d = 0
        for sell in df_hist_daily[index + 1 : ]['最高']:
            d += 1
            df_hist_daily.at[index, f'd{d}'] = round((sell/buy - 1) * 100, 2)
            if d == DAYS:
                break        
        
        #日均
        for mean in DAYS_MEAN:
            sum_pre[mean] += curr_day_close
            if index >= mean:
                sum_pre[mean] -= df_hist_daily.at[index - mean, '收盘']
                df_hist_daily.at[index, f'{mean}日均'] = round(sum_pre[mean]/mean, 2)
            else:
                df_hist_daily.at[index, f'{mean}日均'] = round(sum_pre[mean]/(index + 1), 2)
                
        #print(df_hist_daily.loc[index])

    #debug
    #print(df_hist_daily)   
    df_hist_daily.to_csv(filename_process, index = False)

print(f'\r\nDone, {len(df)}')

601318
Done, 6
